# Fitting Nonlinear Models

If you have a dataset and suspects that the data can be modeled effectively with a linear model, then you typically would use [linear regression](/2_supervised-learning/1_linear-regression) to fit the model. If, however, you think that your data may be best modeled by a *nonlinear* model, then you would typically use a numerical optimization method like gradient descent.

Creating your own models is a important part of science, and PyTorch allows one to easily mix simple algebraic models with neural network models, allowing them to be trained simultaneously as a single optimization problem. In this section, we will discuss how the problem of fitting a model's parameters to data can be solved naturally using gradient descent, construct a PyTorch model of the California Housing Dataset's target variable (median housing price) in terms of its feature variables, construct a PyTorch `Dataset` object for the California Housing Dataset, and finally use the dataset to train the model.

Several of the constructions we learn in this section (in particular, PyTorch's `Module` class, which is used for models, and its `Dataset` class) add a layer of abstraction and complexity to the goals of this lesson that are not strictly necessary for the kind of optimization we are performing here. However, these classes are used identically in a nonlinear model as they are in a convolutional neural network, and they are far more important in such a case. The simple optimization problem examined in this section provides an introduction to the tools we will need in the [next lesson](/4_neural-networks/0_introduction).

## How does fitting nonlinear models to data work?

In the [previous section](/3_pytorch/1_gradient-descent) we saw how gradient descent can be used to find the minimum value of a function with respect to its parameters. Specifically, we saw how we could minimize the value of a function $f(x, y)$ by following the opposite of the function's gradient. Fitting the parameters of a nonlinear model is essentially the same, but we have to separate the loss function from the model function. To explain this, we should start by formalizing some terms and goals:
* Let $f(x, y; a, b, c)$ be the *model* function, which predicts fome value $z = f(x,y; a, b, c)$ in terms of its *inputs*, $x$ and $y$, and its *parameters*, $a$, $b$, and $c$. As an example, let's use a model of a 2D symmetric normal distribution: $f(x, y; \mu_x, \mu_y, \sigma) = \exp\left(\frac{(x - \mu_x)^2 + (y - \mu_y)^2}{2\sigma}\right)$.
* Let our *dataset* contain measured values (with some noise) of both the model's inputs $x$ and $y$ and its associated model output $z$; let there be $n$ rows in the dataset, each equivalent to a tuple $(x_i, y_i, z_i)$ (for $i \in \{0,1,2 ... n-1\})$.
* Let $l_{f}(a,b,c)$ be our loss function, which we are trying to minimize, whose parameters $a$, $b$, and $c$ are identical to those of our model. However, our loss function does not require the model inputs, $x$ and $y$ because it will get samples of $x$ and $y$ from our dataset.

The overall goal of our optimization is to find the values of $\mu_x$, $\mu_y$, and $\sigma$ that minimize the error between $f(x, y; \mu_x, \mu_y, \sigma)$ and the $z$ values observed in our dataset. So, for a row in the dataset $(x_i, y_i, z_i)$, we can define that row's loss to be $\left(f(x_i, y_i; \mu_x, \mu_y, \sigma) - z_i\right)^2$. 

```{note}
We use the function $\left(f(x_i, y_i; \mu_x, \mu_y, \sigma) - z_i\right)^2$ as the loss for a single row $(x_i, y_i, z_i)$ of our dataset. This isn't the only loss we could have used, however. For example, we could have used $\left|f(x_i, y_i; \mu_x, \mu_y, \sigma) - z_i\right|$ or $\log\left|f(x_i, y_i; \mu_x, \mu_y, \sigma) - z_i\right|$; all of these would work because they all have minima when the model's prediction is equal to the measured $z$ value. The different loss functions can have non-trivial effects on an oprimization and and are beyond the scope of this lesson. We will typically use what is called the $L2$ loss function: $(p - m)^2$ where $p$ is the model prediction and $m$ is the measured value from the dataset. The L2 loss function is the most common loss function and is in particular appropriate when the noise in the measurement is normal.
```

Once we've defined the loss for a row, we can define the loss for the whole dataset as the sum of the loss for each row. So we can define our loss function as follows:

$$ l_f(\mu_x, \mu_y, \sigma) = \sum_{i=0}^{n-1} \left(f(x_i, y_i; \mu_x, \mu_y, \sigma) - z_i\right)^2 $$

The above loss function is a little bit more complicated than our previous loss function, but notice that it's independent of there's no reason we can't implement all of this in PyTorch and minimize it just like we did in the previous section.

## Example: The California Housing Dataset

Once again, we'll use the California Housing Dataset as an example for our models. We can start by loading it in.

In [ ]:
import sklearn as skl

# We use scikit-learn to download and return the CA housing dataset:
ca_housing_dataset = skl.datasets.fetch_california_housing()

# Extract the actual data rows and the feature names:
ca_housing_featdata = ca_housing_dataset['data']
ca_housing_featnames = ca_housing_dataset['feature_names']

# We also extract the "target" data, since we are using supervised learning:
ca_housing_targdata = ca_housing_dataset['target']
ca_housing_targnames = ca_housing_dataset['target_names']

As in the previous lessons, we'll split the dataset into train and test subdatasets.

In [ ]:
import numpy as np
# We set a specific random seed to make sure that this notebook runs the same way each time.
np.random.seed(0)

# Randomly select 75% of the rows to be in the training dataset.
all_rows = np.arange(ca_housing_featdata.shape[0])
n_train = int(round(len(all_rows) * 0.75))
n_test = len(all_rows) - n_train
train_rows = np.random.choice(all_rows, n_train, replace=False)
test_rows = np.setdiff1d(all_rows, train_rows)

# Extract these rows into separate matrices:
train_featdata = ca_housing_featdata[train_rows]
train_targdata = ca_housing_targdata[train_rows]
test_featdata = ca_housing_featdata[test_rows]
test_targdata = ca_housing_targdata[test_rows]

### Creating a PyTorch `Dataset` for the California Housing Dataset.

PyTorch has its own class for dealing with datasets. While it isn't strictly necessary for us to use it for this model, it's good practice for more models that process more complex inputs like convolutional neural networks. Defining a dataset type for the CA Housing Dataset lets it interact with other PyTorch tools that we'll see later.

To make a PyTorch dataset, we need to overload the `Dataset` class. This only requires us to write two methods: `__len__` and `__getitem__`; in other words, PyTorch only needs to know how many rows there are in a dataset (`__len__`) and how to get a single row (`__getitem__`). The `__getitem__` method is expected to return a tuple of `(input, output)`, or possibly `(input_1, input_2 ... input_n, output)`.

In [ ]:
import torch
from torch.utils.data import Dataset

# Define a dataset type for the CA Housing Dataset.
# We'll expect the training and test subdatasets to be separate dataset
# objects, so we'll design this class to receive as a parameter the rows of
# the dataset and the targets that it includes.
class CAHousingDataset(Dataset):
    """A PyTorch Dataset implementation of the California Housing Dataset."""
    def __init__(self, features, targets):
        # Convert the features and targets into tensors:
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        # Return (model_input, model_output):
        return (self.features[idx], self.targets[idx])

# Make datasets for test and training data:
train_dset = CAHousingDataset(train_featdata, train_targdata)
test_dset = CAHousingDataset(test_featdata, test_targdata)

In [ ]:
# The datasets should have lengths:
print('train len:', len(train_dset))
print('test len:', len(test_dset))

# Look at one of the entries:
train_dset[10]

### Creating the PyTorch model.

PyTorch also has a class for handling models. It is specifically designed for neural network models, but it is useful for any kind of model, generally. The type is `torch.nn.Module`.

The `Module` type only expects its subtypes to define one method: `forward`, which calculates the model result given a set of inputs. The model parameters should be member variables of the `Module` subtype (collections of parameters like lists of tensors are also fine), and the `Module` will handle various aspects of manipulating these parameters like saving them to or loading them from a file.

The `Module` class does have some idiosyncrasies. First, it expects that you always call the superclass's `__init__` method as the first step in your subclass's `__init__` method. Second, it expects all input tensors (i.e., inputs to the `forward` method) to conform to a particular shape: they must start with dimensions representing **batch** and **channel**.

To explain what the batch and channel are, it's easiest to think of the the inputs for a convolutional neural network (CNN) that is trained using images as inputs. Each image has 3 channels (red, green, and blue), some number of rows ($N$), and some number of columns ($M$). An RGB image would usually be stored in an array or tensor whose shape is `(N, M, 3)`&mdash;i.e., rows, columns, channels. PyTorch wants the first two dimensions to correspond to batch and channel, though, so the shape of an image input to a CNN is expected to be `(B, 3, N, M)` where `B` is the "batch" size. The batch size is the number of samples being computed by the model at once, and so can be any number, including 1. The reason there is a batch size is because for many large models or models that are trained on large inputs, the entire dataset cannot fit in memory at once, and so the loss function cannot be calculated over the whole dataset during one step of the optimizer. Instead it uses a random subset of the dataset during each step. We will see an example of this in the [next lesson](/4_neural-networks/0_introduction).

In the case of our training, the batch size is just the number of rows the model is calculating and the number of channels is just the number of features; each feature is a single number, so instead of `(B, C, N, M)`, we have just `(B, C)`. We will use the entire dataset as a single batch.

The batch size used during training is a hyperparameter of the model training; in theory it should not have a large effect on the optimal point that is found by the optimization algorithm, but it can substantially affect the speed at which the training finds that point.

Let's create a model of the features of the CA Housing Dataset. The dataset has 8 features, so we can use a very simple model with 16 parameters: $c_1$, $c_2$ ... $c_8$ and $q_1$, $q_2$ ... $q_8$. For the 8 features $f_1$, $f_1$, ... $f_8$, we can use the following model $m$:

$$ m(\boldsymbol{f}; \boldsymbol{c}, \boldsymbol{q}) = c_1\,|f_1|^{q_1} + c_2\,|f_2|^{q_2} + ... + c_8\,|f_8|^{q_8} $$

In [ ]:
from torch.nn import Module, Parameter

class CAHousingModel(Module):
    """A model of the median housing price in CA.

    The model has two parameter tensors, each with 8 elements: c and q.
    The model expects inputs with 8 channels each of which is a single real
    number.
    The form of the model is, for each channel f_i:
        sum_{i=1}^8 c_i |f_i|^q_i
    """
    def __init__(self):
        # Start with the superclass init:
        super().__init__()
        # We declare the parameters using the Parameter type.
        # We ccould hoose the initial state as a random number, but for
        # these, we'll use constant guesses (0).
        # Parameters always get requires_grad set to True when they are
        # created, so we don't need to specify it.
        self.c = Parameter(torch.zeros(8))
        self.exp = Parameter(torch.zeros(8))
        # We also have an overall mean parameter:
        self.mean = Parameter(torch.tensor(2.0))
    def forward(self, features):
        # features has shape (B, C): batches, channels.
        (c,q) = (self.c, self.exp)
        return self.mean + torch.sum(c * torch.abs(features)**q, 1)

# Declare a model:
model = CAHousingModel()

Now that we've created a model and a dataset, we can use the dataset to train the model. To do this we use an intermediate class called a `DataLoader`. The `DataLoader` class is in charge of choosing elements from the dataset and feeding them to the model during training. The `DataLoader` is the class that handles the batch sizes, which in our case will just be the size of the dataset. When we iterate over a `DataLoader`, it returns tuples containing batches of inputs and outputs ready to be passed to the model.

In [ ]:
from torch.utils.data import DataLoader

train_dloader = DataLoader(train_dset, batch_size=len(train_dset))
test_dloader = DataLoader(test_dset, batch_size=len(test_dset))

Before we can start minimizing, we also need to declare our loss function. We're using the $L2$ loss function between the model predictions and the measured outputs, so we can use PyTorch's built-in $L2$ loss function, `MSELoss` (though writing a function that calculated $(x - y)^2$ would work just as well). `MSELoss` stands for mean squared error loss. The mean squared error is the sum of the squared errors divided by a constant, so the minimum is the same as the sum of squared errors, which is the $L2$ loss we defined earlier.

In [ ]:
from torch.nn import MSELoss

loss = MSELoss()

We can now set up our training loop. This loop will be almost the same as the minimization loop in the previous section, but instead of just calling a function `f(x,y)` it will call the model to produce a prediction then calculate a loss using our loss function. The loss is what we will minimize.

In [ ]:
# Hyperparameters:
n_steps = 100
lr = 0.25


# Now that we've declared our hyperparameters, let's declare our model.
# This will generate new random starting parameters when run.
model = CAHousingModel()

# Next, we can declare an optimizer.
# We'll continue to use the optimizer SGD. SGD needs to know our model
# parameters, but the `model.parameters()` method will return them for us.
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# Declare our loss function:
loss_fn = MSELoss()

# Now we can take several optimization steps:
for step_number in range(n_steps):
    # In each model we basically do two things: (1) train the model with the
    # training data then (2) test the model with the testing data.
    # When training we want to put the model into training mode.
    model.train()
    # During each step, we will iterate over the training dataloader;
    # because our batch size is the same size as the dataset there will only
    # be one entry to iterate over, but that's fine.
    for (inputs, targets) in train_dloader:
        # We're starting a new step, so we reset the gradients.
        optimizer.zero_grad()
        # Calculate the model prediction for these inputs.
        preds = model(inputs)
        # Calculate the loss between the prediction and the actual outputs.
        train_loss = loss_fn(preds, targets)
        # Have PyTorch backward-propagate the gradients.
        train_loss.backward()
        # Have the optimizer take a step:
        optimizer.step()
        # Print the batch's R2 value:
        var = torch.var(targets)
        train_r2 = 1 - train_loss/var
    # Now that we've finished training, put the model back in evaluation mode.
    model.eval()
    # Evaluate the model using the test data.
    for (inputs, targets) in test_dloader:
        preds = model(inputs)
        test_loss = loss_fn(preds, targets)
        var = torch.var(targets)
        test_r2 = 1 - test_loss/var
    # Print something about this step:
    print(f"Step {step_number:3d}  train r2={train_r2:6.3f}  test r2={test_r2:6.3f}")
# After the optimizer has run, print out what it's found:
print("Final result:")
print(f"  train loss = ", float(train_loss))
print(f"   test loss = ", float(test_loss))

After minimization, our model appears to explain about 34% of the variance of the test dataset. Depending on the dataset one has and the quality of the signal, this could be a high or low fraction of the variance for a particular nonlinear model to explain. For reference, however, our linear model from [Lesson 2](/2_supervised-learning/1_linear-regression) was able to explain about 60% of the variance of the test set.

The fact that our nonlinear model didn't explain as much of the variance as the linear model doesn't actually indicate that the nonlinear model is less powerful or accurate in this case&mdash;in fact, our nonlinear model is strictly more powerful than a simple linear model. Notice that the nonlinear model was not overfit, so its issue was that it did not find a good parameter set for predicting the training data. This likely indicates that it got stuck in a local minimum.

Local minima are not a problem for linear regression, but they are a substantial problem in a lot of nonlinear machine learning. There are many strategies for avoiding them, the most common of which is to try many random starts and use whichever finds the lowest minimum.